In [1]:
"""
regen01.pyからの変更点．
・np.loadで配列読み込み

ストークスパラメータの再構成を行うプログラム．
Planckの観測データCOM_CMB_IQU-nilc_1024_R2.02_full.fitsのI成分をLiteBIRDのTrackで観測していく．
最後の観測データはI=[I_1,I_2,I_3,...I_NPIX]のピクセルに対するIの値とLiteBIRDの重複観測ピクセルI_lu=[,,,I_lu_NPIX]の積
I_observeで
"""
print("import now...")
import numpy as np
import healpy as hp
import matplotlib.pyplot as plt
import astropy.io as ap
import time
import func_stokes as fs
from joblib import Parallel, delayed
from numpy import vectorize as vec


start = time.time()
NSIDE = 256
NPIX = hp.nside2npix(NSIDE)
NPIX
day = 60*60*24#1日の秒数
year = day*365
times = year+1
time_array = np.arange(0,times,1)


orbit_file = "/Users/yusuke/program/py_program/CMB/skymap/regenerate_s/orbit_angle.npz"
orbit = np.load(orbit_file)
orbit = np.array([orbit["theta"][:times], orbit["phi"][:times]])

pix = hp.ang2pix(NSIDE,orbit[0],orbit[1])

"""Planckのマップをreadして解析する"""
file_path = "/Users/yusuke/program/py_program/CMB/skymap/data/LFI_SkyMap_030-BPassCorrected_0256_R2.01_full.fits"
I_planck = hp.read_map(file_path, field = 0, dtype=np.float32)#PlanckのRING型データ
I_obs = I_planck[pix]#Planckのデータを観測されるpix順に並び換えた時系列観測データI_obs
I_map = np.zeros(NPIX)
I_map[pix[:]] = I_planck[pix]


import now...
NSIDE = 256
ORDERING = NESTED in fits file
INDXSCHM = IMPLICIT
Ordering converted to RING


In [230]:
def detec(t, l):
    det = np.array([np.ones(l),np.sin(2*w*t), 0,np.cos(2*w*t)])
    return det

def mat(t, l):
    D_mat = np.array([
        [np.array([l]), np.sin(2*w*t).sum(),0,np.cos(2*w*t).sum()],
        [np.sin(2*w*t).sum(), (np.sin(2*w*t)**2).sum(), 0, (np.cos(2*w*t)*np.sin(2*w*t)).sum()],
        [0,0,0,0],
        [np.cos(2*w*t).sum(), (np.cos(2*w*t)*np.sin(2*w*t)).sum(), 0,(np.cos(2*w*t)*np.cos(2*w*t)).sum()]
        ])
    return D_mat

In [237]:
w = 2*np.pi*(0.3/60)
t = []
length = []
p_i = []
for i in range(2):
    print(i)
    t.append(np.where(pix == i)[0])
    length.append(len(t[i]))
    p_i.append(I_obs[t[i]])

t = np.array(t)

l = np.array(length)
#l=length
print(l)
p_i = np.array(p_i)

f = np.vectorize(detec)
g = np.vectorize(mat)
det = f(t,l)
D_mat = g(t,l)

D_mat

0
1
[83 90]


array([[array([83]), -2.0970225577834136, 0, 67.49123223112313],
       [-2.0970225577834136, 27.832780215481954, 0, -2.1172260259376214],
       [0, 0, 0, 0],
       [67.49123223112313, -2.1172260259376214, 0, 55.16721978451806]],
      dtype=object)

In [238]:
sum_D_pinv = np.linalg.pinv(D_mat)

LinAlgError: 1-dimensional array given. Array must be at least two-dimensional

In [4]:
sum_D_pinv = np.linalg.pinv(D_mat.sum(2))
D_E = np.dot(D_mat.sum(2), sum_D_pinv)
wp = det*I_obs[t]
sum_wp = wp.sum(1)
s_tilde = np.dot(sum_D_pinv, sum_wp)

AxisError: axis 2 is out of bounds for array of dimension 1